In [18]:
from utils.load_data import RequestData, RequestBusData, DataPreprocess
from utils.columns import Columns


import pandas as pd
import os

# warning 무시
import warnings
warnings.filterwarnings(action='ignore')

from tqdm import tqdm

In [5]:
# 모듈 수정할 때 사용
import importlib
# importlib.reload(utils.db_connector)

pd.set_option("display.max_columns", 50)

In [3]:
# MySQL 데이터 적재

In [16]:
# db정보 들어있는 텍스트파일
db_info_path = 'secret_key/db_info.txt'
db_info_dict = {}

with open(db_info_path, 'r') as f:
    for line in f.readlines():
        line_info = line.split('=')
        key = line_info[0].strip()
        val = line_info[1].strip()
        
        db_info_dict[key] = val
        

In [21]:
db_info_dict['database']

'localdata_test'

In [24]:
# Mysql db 연결
from utils.db_connector import DBManagement


# localdata 데이터베이스에 connection
dbm = DBManagement(
    **db_info_dict
)

print(f'성공적으로 MySQL {db_info_dict["database"]} 데이터베이스에 연결 완료')

성공적으로 MySQL localdata_test 데이터베이스에 연결 완료


## 1.1 bulk형 csv파일 적재
- 각 인허가 csv파일에 대해 로드 -> 전처리 -> Mysql 적재과정

In [25]:
# 파일 경로 검색
# 절대경로
path = os.getcwd()

csv_path = os.getcwd() + '/csv_data'
in_csv_files = sorted(os.listdir(csv_path))

try:
    in_csv_files.remove('.DS_Store')
except:
    pass

in_csv_files

['01_hospital.csv',
 '02_pharmacy.csv',
 '03_laundry.csv',
 '04_hair.csv',
 '05_gym.csv',
 '06_mart.csv',
 '07_cafe.csv',
 '08_convenience.csv']

In [26]:
columns_dict = {
                '개방자치단체코드': 'opnSfTeamCode',
                '관리번호': 'mgtNo',
                '개방서비스아이디': 'opnSvcId',
                '데이터갱신구분': 'updateGbn',
                '데이터갱신일자': 'updateDt',
                '사업장명': 'bplcNm',
                '소재지우편번호': 'sitePostNo',
                '소재지전체주소': 'siteWhlAddr',
                '도로명우편번호': 'rdnPostNo',
                '도로명전체주소': 'rdnWhlAddr',
                '영업상태구분코드': 'trdStateGbn',
                '상세영업상태코드': 'dtlStateGbn',
                '좌표정보(X)': 'x',
                '좌표정보(Y)': 'y',
                '최종수정시점': 'lastModTs',
                '업태구분명': 'uptaeNm',
               }

# 영문 column(API 요청할때나 MySQL 적재할 때 사용)
cols = Columns(columns_dict)

# MySQL에서 적재할 때, POINT type으로 추가로 위경도 저장하기 때문에
cols_eng = cols.eng + ['coordinates']
cols_eng

['opnSfTeamCode',
 'mgtNo',
 'opnSvcId',
 'updateGbn',
 'updateDt',
 'bplcNm',
 'sitePostNo',
 'siteWhlAddr',
 'rdnPostNo',
 'rdnWhlAddr',
 'trdStateGbn',
 'dtlStateGbn',
 'x',
 'y',
 'lastModTs',
 'uptaeNm',
 'coordinates']

In [27]:
# MySQL 저장코드
for file_name in in_csv_files:


    # 파일 경로
    file_path = csv_path + '/' + file_name

    # 데이터 로드
    data_df = RequestData.get_csvdata(path = file_path)

    # 전처리
    data_preprocess_df = DataPreprocess.preprocess(data_df, columns_dict, is_bulk=True)
    
    # data_preprocess_df.sample(n=1000).to_csv(f'ex_{file_name[3:-4]}_5174.csv', index=False, encoding='cp949')
    # table 생성
    table_name = file_name[3:-4]
    dbm.create_table(table_name)
    dbm.commit()
    
    # DB에 데이터 젛기
    for _, row in tqdm(data_preprocess_df.iterrows(), mininterval=0.01):


        dbm.update_record(row, cols_eng)

    # 완료되면 commit
    dbm.commit()
    

    print(f"{file_name} 작업 완료")

data_preprocess_df.head()

69935it [00:14, 4922.13it/s]


01_hospital.csv 작업 완료


24046it [00:05, 4800.97it/s]


02_pharmacy.csv 작업 완료


20784it [00:04, 4955.31it/s]


...

50598it [00:16, 3101.15it/s]


07_cafe.csv 작업 완료


52052it [00:16, 3252.96it/s]


08_convenience.csv 작업 완료


,opnSfTeamCode,mgtNo,opnSvcId,updateGbn,updateDt,bplcNm,sitePostNo,siteWhlAddr,rdnPostNo,rdnWhlAddr,trdStateGbn,dtlStateGbn,x,y,lastModTs,uptaeNm,lat,lon
0,3000000,2010300012905600099,11_43_02_P,I,2018-08-31 23:59:59,씨유 종로원서점,110-280,서울특별시 종로구 원서동 45번지 1층,NULL,"서울특별시 종로구 창덕궁길 120 (원서동,1층)",1,0,198988.459597,453392.779978,2012-07-19 17:29:20,NULL,126.989336,37.582836
1,3000000,2011300012905600061,11_43_02_P,I,2018-08-31 23:59:59,세븐일레븐 종로인의점,110-410,서울특별시 종로구 인의동 31번지,NULL,서울특별시 종로구 종로31길 27 (인의동),1,0,199888.381723,452197.706939,2017-02-06 09:25:59,NULL,126.999524,37.572069
2,3000000,2019300020205600048,11_43_02_P,I,2019-07-11 02:21:33,이마트24종로마루점,NULL,서울특별시 종로구 숭인동 211번지 7호 흥남빌딩,03115,"서울특별시 종로구 난계로27길 44-7, 흥남빌딩 (숭인동)",1,0,201744.215856,452328.641292,2019-07-09 10:01:09,NULL,127.020532,37.573247
5,3000000,2019300020205600053,11_43_02_P,I,2019-08-15 02:22:17,씨유 종로관훈점,NULL,서울특별시 종로구 관훈동 198번지 17호 일정빌딩,03149,"서울특별시 종로구 인사동7길 37, 일정빌딩 1층 (관훈동)",1,0,198574.909900,452264.246969,2019-08-13 17:11:38,NULL,126.984656,37.572667
7,3000000,2019300020205600038,11_43_02_P,I,2019-06-14 02:21:30,GS25종로로얄점,NULL,서울특별시 종로구 수송동 85번지 서머셋팰리스서울,03143,"서울특별시 종로구 율곡로2길 7, 서머셋팰리스서울 1층 1호 (수송동)",1,0,198271.843790,452533.899674,2019-06-12 09:02:55,NULL,126.981225,37.575096


In [10]:
# API에서 호출할 때, 파라미터 이름으로 쓰이는 서비스 이름 가져오는 함수
def get_request_service_names(path, csv_name):
    file_path = f"{path}/excel_data/{csv_name[:-4]}"
    service_names = os.listdir(file_path)
    
    try:
        service_names.remove('.DS_Store')
    except:
        pass
    service_names = [ name[:-5] for name in service_names]
    return service_names